In [81]:
# imports
import pandas as pd
import requests
import os # use this to access your environment variables
FOURSQUARE_KEY = os.getenv('FOURSQUARE_KEY')
YELP_KEY = os.getenv('YELP_KEY')
%store -r bike_station_information_df
data = bike_station_information_df



# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [44]:
def get_venues_fs(latitude, longitude, radius, api_key, categories):
    url = "https://api.foursquare.com/v3/places/search"
    params = {
        'll': f"{latitude},{longitude}",
        'radius': radius,
        'v': '20220101',  
    }

    headers = {
        "Accept": "application/json",
        "Authorization": api_key
    }

    if categories:
        params['categoryId'] = categories

    response = requests.request("GET", url, params=params, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        return response.json()


radius=1000
categories=None

# Initialize a dictionary to store results
result_dict = {}

# Function to process each row and save the result
def process_row(row):
    result = get_venues_fs(row['Latitude'], row['Longitude'], radius, FOURSQUARE_KEY, categories)
    result_dict[row.name] = result

# Apply the function to the entire DataFrame
data.apply(process_row, axis=1)

# Display the result dictionary
print(result_dict)

{0: {'results': [{'fsq_id': 'f78b51e353f14b341a813987', 'categories': [{'id': 12009, 'name': 'Education', 'short_name': 'Education', 'plural_name': 'Education', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'LikelyOpen', 'distance': 259, 'geocodes': {'main': {'latitude': 49.032584, 'longitude': 2.066059}, 'roof': {'latitude': 49.032584, 'longitude': 2.066059}}, 'link': '/v3/places/f78b51e353f14b341a813987', 'location': {'address': '32 boulevard du Port', 'admin_region': 'Île-de-France', 'country': 'FR', 'formatted_address': '32 boulevard du Port, 95000 Cergy', 'locality': 'Cergy', 'postcode': '95000', 'region': 'Île-de-France'}, 'name': 'Ecole de biologie industrielle', 'related_places': {}, 'timezone': 'Europe/Paris'}, {'fsq_id': '3088c19d782d403ff66ff751', 'categories': [{'id': 17044, 'name': 'Fashion Accessories Store', 'short_name': 'Fashion Accessories Store', 'plural_name': 'Fashion Accessories St

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [71]:
parsed_fourSquare_data = {}

for key, value in result_dict.items():
  venues_data = value.get('results', [])
  parsed_venues = []

  for venue in venues_data:
    venue_name = venue.get('name', 'N/A')
    category_info = venue.get('categories', [])
        
    if category_info:
        category_name = category_info[0].get('name', 'N/A')
    else:
      category_name = 'N/A'
        
    distance = venue.get('distance', 'N/A')

    # Create a dictionary with the extracted information
    parsed_venue = {
      'name': venue_name,
      'category': category_name,
      'distance': distance
    }

    parsed_venues.append(parsed_venue)

  # Store the parsed information in the new dictionary
  parsed_fourSquare_data[key] = parsed_venues

# Display the parsed data
for key, value in parsed_fourSquare_data.items():
  print(f"Key: {key}")
  print(value)

Key: 0
[{'name': 'Ecole de biologie industrielle', 'category': 'Education', 'distance': 259}, {'name': 'Esprit Clean', 'category': 'Fashion Accessories Store', 'distance': 283}, {'name': 'Le Millésime du Port', 'category': 'French Restaurant', 'distance': 634}, {'name': 'La Taverne des Rois', 'category': 'Restaurant', 'distance': 648}, {'name': 'Au Fourmont Village', 'category': 'Bakery', 'distance': 359}, {'name': "Moulin Vert Lava'9", 'category': 'N/A', 'distance': 391}, {'name': 'SAN MARINA Cergy', 'category': 'N/A', 'distance': 394}, {'name': 'École Maternelle Publique le Village', 'category': 'Elementary School', 'distance': 397}, {'name': 'France Depannage Auto', 'category': 'Miscellaneous Store', 'distance': 425}, {'name': 'Pharmacie des Trois Gares', 'category': 'Drugstore', 'distance': 448}]
Key: 1
[{'name': "O'Sullivans Pub Compagny", 'category': 'Beer Bar', 'distance': 60}, {'name': 'Bosphore Restauration', 'category': 'Pizzeria', 'distance': 149}, {'name': 'City Rock', 'cat

Put your parsed results into a DataFrame

In [101]:
# Create an empty list to store dictionaries
data_list = []

# Iterate through each key in the dictionary
for key in parsed_fourSquare_data:
  # Extract the list of results from the dictionary
  results_list = parsed_fourSquare_data[key]
    
  # Iterate through each result in the list
  for result in results_list:
    # Extract relevant information
    name = result.get('name', 'N/A')
    category = result.get('category', 'N/A')
    distance = result.get('distance', 'N/A')

    #Append the data to the list as a dictionary, including the key
    data_list.append({'Key': key, 'Name': name, 'Category': category, 'Distance': distance})

# Create a DataFrame from parsed_fourSquare_data
fourSquare_result_df = pd.DataFrame(data_list)

# Display the DataFrame
fourSquare_result_df

,Key,Name,Category,Distance
0,0,Ecole de biologie industrielle,Education,259
1,0,Esprit Clean,Fashion Accessories Store,283
2,0,Le Millésime du Port,French Restaurant,634
3,0,La Taverne des Rois,Restaurant,648
4,0,Au Fourmont Village,Bakery,359
...,...,...,...,...
435,43,Ecole la Sébille,Elementary School,461
436,43,Château de Gency,Event Space,489
437,43,Koezio,Arts and Entertainment,894
438,43,Stanley Serrurier Securite,Locksmith,546


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [108]:
# API constants
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.
SEARCH_LIMIT = 10


def get_venues_yelp(latitude, longitude, radius, api_key):
    url = f"{API_HOST}{SEARCH_PATH}"
    
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'radius': radius,
        'categories': '',
        'sort_by': 'best_match',
        'limit': SEARCH_LIMIT
    }

    headers = {
        'Authorization': f'Bearer {api_key}',
        'accept': 'application/json',
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        return 'Not Successful'

 
radius = 1000

# Initialize a dictionary to store results
yelp_result_dict = {}

# Function to process each bike shop location and save the result
def yelp_process_row(row):
    yelp_result = get_venues_yelp(row['Latitude'], row['Longitude'], radius, YELP_KEY)
    yelp_result_dict[row.name] = yelp_result

# Apply the function to the entire bikeshop data
data.apply(yelp_process_row, axis=1)

# Display the result dictionary
yelp_result_dict

{0: {'businesses': [{'id': 'hZptG_J3jyen5SHTEhi_cA',
    'alias': 'la-rotisserie-ô-cergy-3',
    'name': 'La Rotisserie Ô',
    'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/QHsSynNQiqTpS3NxaqULfg/o.jpg',
    'is_closed': False,
    'url': 'https://www.yelp.com/biz/la-rotisserie-%C3%B4-cergy-3?adjust_creative=RgThseK9o5yqgju5doYPQw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=RgThseK9o5yqgju5doYPQw',
    'review_count': 7,
    'categories': [{'alias': 'french', 'title': 'French'},
     {'alias': 'fondue', 'title': 'Fondue'},
     {'alias': 'desserts', 'title': 'Desserts'}],
    'rating': 4.5,
    'coordinates': {'latitude': 49.032665, 'longitude': 2.061423},
    'transactions': [],
    'price': '€€',
    'location': {'address1': '3 rue de Neuville',
     'address2': None,
     'address3': None,
     'city': 'Cergy',
     'zip_code': '95000',
     'country': 'FR',
     'state': '95',
     'display_address': ['3 rue de Neuville', '95000 Cergy', 'France']}

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [107]:
parsed_yelp_data = {}

for key, value in yelp_result_dict.items():
  yelp_venues_data = value.get('businesses', [])
  yelp_parsed_venues = []

  for venue in yelp_venues_data:
    yelp_venue_name = venue.get('name', 'N/A')
    category_info = venue.get('categories', [])
    
    if category_info:
        category_title = category_info[0].get('title', 'N/A')
    else:
        category_title = 'N/A'
    
    distance = venue.get('distance', 'N/A')
    rating = venue.get('rating', 'N/A')

    # Create a dictionary with the extracted information
    yelp_parsed_venue = {
        'name': yelp_venue_name,
        'category': category_title,
        'distance': distance,
        'rating': rating
    }

    yelp_parsed_venues.append(yelp_parsed_venue)

  # Store the parsed information in the new dictionary
  parsed_yelp_data[key] = yelp_parsed_venues

# Display the parsed data
for key, value in parsed_yelp_data.items():
    print(f"Key: {key}")
    print(value)


Key: 0
[{'name': 'La Rotisserie Ô', 'category': 'French', 'distance': 500.0150705429588, 'rating': 4.5}, {'name': 'Hippopotamus', 'category': 'Steakhouses', 'distance': 592.8515667925261, 'rating': 2.5}, {'name': 'La Cavallina', 'category': 'Italian', 'distance': 496.28838666100484, 'rating': 5.0}, {'name': "Les Relais d'Alsace", 'category': 'French', 'distance': 638.7411611969612, 'rating': 4.0}, {'name': "O'Sullivans", 'category': 'Pubs', 'distance': 823.3760717014171, 'rating': 3.0}, {'name': 'Ici et Ailleurs', 'category': 'French', 'distance': 1121.5154527868633, 'rating': 4.0}, {'name': "Le Port d'Asie", 'category': 'Thai', 'distance': 495.94209413674236, 'rating': 4.0}, {'name': 'Au festival de la Moule', 'category': 'Seafood', 'distance': 518.4645316945345, 'rating': 3.0}, {'name': "Relais d'alsace", 'category': 'French', 'distance': 625.8610766353718, 'rating': 3.0}, {'name': 'Le Latino', 'category': 'Mexican', 'distance': 489.92476629982076, 'rating': 1.0}]
Key: 1
[{'name': "O

Put your parsed results into a DataFrame

In [109]:
# Create an empty list to store dictionaries
yelp_data_list = []

# Iterate through each key in the dictionary
for key in parsed_yelp_data:
  # Extract the list of results from the dictionary
  yelp_results_list = parsed_yelp_data[key]
    
  # Iterate through each result in the list
  for result in yelp_results_list:
    # Extract relevant information
    name = result.get('name', 'N/A')
    category = result.get('category', 'N/A')
    distance = result.get('distance', 'N/A')
    rating = result.get('rating', 'N/A')

    #Append the data to the list as a dictionary, including the key
    yelp_data_list.append({'Key': key, 'Name': name, 'Category': category, 'Distance': distance, 'Rating': rating})

# Create a DataFrame from parsed_yelp_data
yelp_result_df = pd.DataFrame(yelp_data_list)

yelp_result_df

,Key,Name,Category,Distance,Rating
0,0,La Rotisserie Ô,French,500.015071,4.5
1,0,Hippopotamus,Steakhouses,592.851567,2.5
2,0,La Cavallina,Italian,496.288387,5.0
3,0,Les Relais d'Alsace,French,638.741161,4.0
4,0,O'Sullivans,Pubs,823.376072,3.0
...,...,...,...,...,...
404,42,La Fontaine de Trévi,French,123.062331,3.5
405,42,Uchitomi,Japanese,298.866798,4.0
406,43,Ici et Ailleurs,French,442.971295,4.0
407,43,La Ciboulette,French,484.855675,4.5


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

YELP API provided more complete data. YELP api provided ratings and more information on the location that foursquare failed to provide such as rating, price review count, aliases and more. 

Get the top 10 restaurants according to their rating

In [111]:
top_10_restaurants = yelp_result_df.sort_values(by='Rating', ascending=False).head(10)
print('Top 10 Restaurants according to rating')
print(top_10_restaurants)

Top 10 Restaurants according to rating
     Key              Name     Category     Distance  Rating
204   20        Misterpizz        Pizza   745.728592     5.0
194   19         City Rock      Lounges   658.711245     5.0
203   20        Le Factory       French   560.202395     5.0
214   22            Subway    Fast Food   416.017996     5.0
217   22        Le Factory       French   641.535240     5.0
78     7         City Rock      Lounges   582.002629     5.0
77     7  Dans Les Etoiles       French   744.369110     5.0
76     7      La Cavallina      Italian   894.242736     5.0
75     7           Le Club  Restaurants  1078.741685     5.0
219   22        Misterpizz        Pizza  1076.786017     5.0
